In [ ]:
import nltk
import pandas as pd
import stanza
from nltk.corpus import stopwords

In [ ]:
file_path = "../../data/adhd-beliefs-pt/adhd-beliefs-pt-prepared.pkl"
df = pd.read_pickle(file_path)
df.head()

In [ ]:
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))

In [ ]:
stanza.download("pt")
nlp = stanza.Pipeline(lang="pt", processors="tokenize,mwt,pos,lemma")

In [ ]:
def preprocess_text(text: str) -> str:
    doc = nlp(text.lower())
    tokens = []
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma
            if lemma.isalpha() and lemma not in stopwords_pt:
                tokens.append(lemma)
    return " ".join(tokens)

In [ ]:
text_columns = ["special_interest", "diary_entry", "selfdefining_memory", "empty_sheet"]

for column in text_columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].apply(preprocess_text)

In [ ]:
temp_df = df[text_columns].replace("nan", "").fillna("")
df["merged_text"] = temp_df.agg(" ".join, axis=1)
df["merged_text"] = df["merged_text"].replace("   ", "nan")

In [ ]:
file_path = "../../data/adhd-beliefs-pt/adhd-beliefs-pt-preprocessed.pkl"
df.to_pickle(file_path)
print("Text preprocessing completed and saved to", file_path)

In [ ]:
df